In [16]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import folium
import seaborn as sns
import missingno as msno

In [17]:
%config InlineBackend.figure_format = 'retina'
#나눔 글자체 설치 
!apt -qq -y install fonts-nanum
 

fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [18]:
import matplotlib.font_manager as fm
#나눔고딕 글자체의 전체 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
#글자체의 경로와 글자 크기 설정
font = fm.FontProperties(fname=fontpath, size=9)


In [19]:
plt.rc('font', family='NanumBarunGothic') 
#폰트 업데이트
mpl.font_manager._rebuild()

In [14]:
card = pd.read_csv("/content/card_20200717.csv", encoding="utf-8")

In [15]:
column_name = ["receipt_dttm", "mrhst_induty_cl_nm", "salamt"]

In [20]:
card = card[column_name]
#필요없는 column 삭제

In [21]:
sum_by_date_industry = card["salamt"].groupby([card["receipt_dttm"], card["mrhst_induty_cl_nm"]]).sum()
sum_by_date_industry = sum_by_date_industry.reset_index() 
#salamt데이터를 receipt_dttm와 mrhst_induty_cl_nm기준으로 더함

In [22]:
for i, date in enumerate(sum_by_date_industry["receipt_dttm"]):

  if (20200100 <=  date <= 20200131) :
    sum_by_date_industry.loc[i,"month"] = "1월"
  elif (20200200 <=  date <= 20200229) :
    sum_by_date_industry.loc[i,"month"] = "2월"
  elif (20200300 <=  date <= 20200331) :
    sum_by_date_industry.loc[i,"month"] = "3월"
  elif (20200400 <=  date <= 20200430) :
    sum_by_date_industry.loc[i,"month"] = "4월"
  elif (20200500 <=  date <= 20200531) :
    sum_by_date_industry.loc[i,"month"] = "5월"
  else:
    sum_by_date_industry.loc[i,"month"] = "6월"
#receipt_dttm 기준으로 월을 나눔

In [23]:
sum_by_date_industry_new = sum_by_date_industry["salamt"].groupby([sum_by_date_industry["month"], sum_by_date_industry["mrhst_induty_cl_nm"]]).sum()
sum_by_date_industry_new = sum_by_date_industry_new.reset_index() 
sum_by_date_industry_new
#월별로 salamt를 다시 더하여 총 salamt를 구함

,month,mrhst_induty_cl_nm,salamt
0,1월,1급 호텔,960082843
1,1월,2급 호텔,988724460
2,1월,CATV,11803740
3,1월,CATV홈쇼핑,21671234046
4,1월,L P G,1135281474
...,...,...,...
1527,6월,화 랑,239127960
1528,6월,화 원,1886794674
1529,6월,화 장 품,11915542902
1530,6월,화물 운송,1940681565


In [24]:
sum_by_date_industry_new.rename(columns={"mrhst_induty_cl_nm":"column_name"}, inplace=True)
#column 이름 바꿈

In [25]:
card = sum_by_date_industry_new

In [26]:
card["column_name"] = card["column_name"].str.replace(' ','')
#띄워쓰기 삭제

In [27]:
card_travel = card[card["column_name"].str.contains(
    "고속버스|관광여행|기타교통수단|렌트카|여객선|콘도|항공사|철도")]
#여행관련 데이터 합

In [28]:
card_hotel = card[card["column_name"].str.contains(
    "1급호텔|2급호텔|특급호텔")]

In [29]:
card_shop = card[card["column_name"].str.contains(
    "기념품점|면세점")]

In [30]:
card_culture = card[card["column_name"].str.contains(
    "기타레져업|노래방|레져업소(회원제형태)|문화취미기타|볼링장|수영장|수족관|스키장|스크린골프|영화관|종합레져타운|화랑")]

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [31]:
card_travel = pd.DataFrame(card_travel["salamt"].groupby(card_travel["month"]).sum())
#월을 기준으로 salamt데이터 값을 모두 더함

In [32]:
card_hotel = pd.DataFrame(card_hotel["salamt"].groupby(card_hotel["month"]).sum())
card_shop = pd.DataFrame(card_shop["salamt"].groupby(card_shop["month"]).sum())
card_culture = pd.DataFrame(card_culture["salamt"].groupby(card_culture["month"]).sum())

In [33]:
card_travel.rename(columns={"salamt":"travel salamt"}, inplace=True)
card_hotel.rename(columns={"salamt":"hotel salamt"}, inplace=True)
card_shop.rename(columns={"salamt":"duty free salamt"}, inplace=True)
card_culture.rename(columns={"salamt":"culture salamt"}, inplace=True)
#열의 이름을 바꿔줌

In [34]:
card = pd.concat([card_travel, card_hotel, card_shop, card_culture], axis=1)
#데이터 병합

In [35]:
card

,travel salamt,hotel salamt,duty free salamt,culture salamt
month,,,,
1월,106430009884,16524218913,20909897499,26791377824
2월,38577367125,19259270028,13205138761,20888978865
3월,7994933016,11955026851,5248790423,19502255333
4월,14790332009,11919468773,1004988398,17558786035
5월,33468107237,17461426602,1237682393,22763949709
6월,16720891873,10072392179,528929941,9667517470


In [36]:
card.drop("6월", inplace=True)

In [37]:
!pip install plotly

In [38]:
!pip install plotly --upgrade

     |████████████████████████████████| 12.9MB 360kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [39]:
! pip install cufflinks

In [40]:
! pip install chart_studio

     |████████████████████████████████| 71kB 1.8MB/s 


In [41]:
import cufflinks as cf
cf.go_offline(connected = True)

In [42]:
import chart_studio 
chart_studio.tools.set_credentials_file(username = 'eee5609', api_key='mPNTRALsmhmSo7pbQQh4')

In [43]:
import chart_studio.plotly as py
import plotly
import plotly.graph_objects as go
import plotly.express as px

plotly.offline.plot
#필요한 라이브러리 및 파일 설치

<function plotly.offline.offline.plot>

In [44]:
fig_decrease = px.line(card, x=card.index, y=["travel salamt", "hotel salamt", "duty free salamt", "culture salamt"])

In [45]:
py.iplot(fig_decrease)

In [ ]:
#여행:  고속버스, 관광여행, 기타교통수단, 렌트카,  여객선, 콘도, 항공사, 철도
#호텔: 특급호텔, 1급호텔, 2급호텔
#면세점:  기념품점,  면세점
#문화시설:  기타레져업, 노래방, 레져업소(회원제형태), 
#          문화취미기타, 볼링장, 수영장, 수족관, 스키장, 스크린골프, 영화관, 종합레져타운, 화랑